In [ ]:
# Class FFNN19

class FFNN19:
    def __init__(self,
                 hidden_layer_count=1,
                 input_neuron_count=1,
                 output_neuron_count=1,
                 ):
        
        self.hidden_layer_count             = hidden_layer_count; # banyak hidden layer
        self.input_neuron_count             = input_neuron_count; # banyak neuron pada layer input
        self.output_neuron_count            = output_neuron_count;  # banyak neuron pada layer output
        self._weight_layer_count            = hidden_layer_count+1; # baca[1]
        self.weight_layers                  = [Neuron(0) for _ in range(self._weight_layer_count)]; # baca[1]
        self.hidden_layers                  = [Neuron(0) for _ in range(hidden_layer_count)]; # List berisi matrix penyimpan angka dari setiap neuron pada hidden layer (bias=1 juga termasuk di sini)

        # _activated_hidden_layers mungkin akan dibuang jika Class Neuron dapat menyimpan history terlepas
        # dari tempat neuron di dalam matrix sudah direplace dengan angka lain 
        self._activated_hidden_layers       = [Neuron(0) for _ in range(hidden_layer_count)]; # Sama dengan hidden_layers tetapi sudah diubah dengan fungsi aktivasi
        self._output_layer                  = [Neuron(0) for _ in range(output_neuron_count)]; # Neuron berisi output untuk 1x pass feedforward
        self._neuron_layer_set              = False; # Flag untuk menentukan apakah banyak neuron(width) setiap layer sudah diset, dipakai saat inisialisasi weight
        self._weight_layers_initialized     = False; # Flag untuk menentukan apakah weight sudah diinisialisasi, dipakai saat memanggil _feedforward()
        self.outputs                        = [] # Baru keinget kalau input bisa batch, jadi setiap kali pass feedforward akan disimpan (bisa berubah kalau ada implementasi lebih bagus)


# Note:
# [1]: Di materi memang tidak ada weight layer, tetapi butuh representasi dari setiap edge graph
# kalau misal coba print(modelInstance.weight_layers[0]), akan tampak output
# [[0 1 3 1 3 1] <- row-0
#  [0 3 3 3 3 3]
#  [0 3 1 2 2 2]
#  [0 1 1 1 3 1]
#  [0 2 2 3 1 3]]
#   ^
#   col-0
# satu kolom (ke bawah) itu untuk weight menuju satu node di next layer
# satu row (ke samping) itu source nodenya
# jadi untuk baris ke-0, itu weight dari bias ke node 0,1,2,3,4,5
# untuk col-0 berarti menuju bias di next layer (nilai x bias selalu 1, yg berubah weightnya)

In [ ]:
# Prosedur setHiddenLayerNeuronCount(list_of_neuron_per_layer: list)
# Desc: Untuk menentukan jumlah neuron (width) setiap layer
# contoh: modelInstance.setHiddenLayerNeuronCount([1,7,3,4])
# layer 1: 1 neuron
# layer 2: 7 neuron
# layer 3: 3 neuron
# layer 4: 4 neuron

def setHiddenLayerNeuronCount(self,*args):
    # Periksa jika args panjangnya nol, kalau nol berarti input via terminal satu per satu sesuai banyak layer
    if len(args)==0:
        for i in range(self.hidden_layer_count):
            # Kenapa +1? Karena ada bias, dan ini agak botching karena baru keinget di akhir
            self.hidden_layers[i] = self.neuronGenerator(np.zeros,(input())+1)

            # Nilai bias selalu 1
            self.hidden_layers[i][0] = Neuron(1)

        # Semacam flag yang dipake di tempat lain, untuk menentukan apakah jumlah layer tiap neuron sudah diset atau belum
        self._neuron_layer_set    = True

    # Langsung pake list
    elif len(args)==1:
        if len(args[0])!=self.hidden_layer_count:
            # Kalau banyak elemen list beda sama hidden layer countnya
            raise ValueError(f"Layer count mismatch: expected {self.hidden_layer_count} hidden layer, but {len(args[0])} were given")
        else:
            # Iterasi sebanyak hidden layer, lalu set setiap layer dengan angka yg dimasukkan
            # nilai pertama adalah bias
            # misalkan setHiddenLayerNeuronCount([3,7])
            # hidden layer ke-1: [1,0,0,0]
            # hidden layer ke-2: [1,0,0,0,0,0,0,0]
            for i in range(self.hidden_layer_count):
                self.hidden_layers[i] = self.neuronGenerator(np.zeros,args[0][i]+1)
                self.hidden_layers[i][0] = Neuron(1)
    
            # Semacam flag yang dipake di tempat lain, untuk menentukan apakah jumlah layer tiap neuron sudah diset atau belum
            self._neuron_layer_set = True
    else:
        # Kalau ngisi argumen kebanyakan
        raise TypeError(f"FFNN19.setHiddenLayerNeuronCount() takes exactly 1 positional arguments but {len(args)} were given")

In [ ]:
# Prosedur inisialisasi weight
# Sebenernya tiga kode ini sama dan boilerplate, yg beda cuma fungsinya, masih bisa diimprove dijadiin 1 method
    
def uniformWeightDistribution(self,lower,upper,seed):
    if self._neuron_layer_set:
        rng = np.random.default_rng(seed)
        for i in range(self._weight_layer_count):
            # Kalau iterasi pertama, banyak edge dari input layer ke hidden layer pertama adalah banyak neuron input dikali banyak hidden layer pertama
            if i==0:
                self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(self.input_neuron_count+1,len(self.hidden_layers[i])))
            # Kalau iterasi terakhir, banyak edge dari hidden layer terakhir ke output adalah banyak neuron hidden layer terakhir dikali banyak neuron output
            elif i==self._weight_layer_count-1:
                self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),self.output_neuron_count))
            # Banyak edge weight untuk antar hidden layer
            else:
                self.weight_layers[i] = self.neuronGenerator(rng.uniform,lower,upper,size=(len(self.hidden_layers[i-1]),len(self.hidden_layers[i])))
            # Khusus untuk uniform dan normal, karena ada bias, weight menuju bias dibuat 0 karena harusnya nilai bias tidak berubah, yg berubah weightnya
            # Karena di layer output tidak ada bias, dijadikan pengecualian.
            if i!=self._weight_layer_count-1:
                self.weight_layers[i][:,0] = Neuron(0)
        self._weight_layers_initialized     = True;
    else:
        raise ValueError(f"Neuron layer count not set, please set neuron layer count before weight initialization")